In [0]:
dbutils.widgets.text(
    "silver_path",
    "dbfs:/Volumes/workspace/ecommerce/ecommerce_data/medallion/silver/events"
)

dbutils.widgets.text(
    "gold_path",
    "dbfs:/Volumes/workspace/ecommerce/ecommerce_data/medallion/gold/products"
)


In [0]:
from pyspark.sql import functions as F

silver_path = dbutils.widgets.get("silver_path")
gold_path = dbutils.widgets.get("gold_path")

silver = spark.read.format("delta").load(silver_path)

product_perf = (silver.groupBy("product_id")
    .agg(
        F.countDistinct(F.when(F.col("event_type")=="view", F.col("user_id"))).alias("views"),
        F.countDistinct(F.when(F.col("event_type")=="purchase", F.col("user_id"))).alias("purchases"),
        F.sum(F.when(F.col("event_type")=="purchase", F.col("price")).otherwise(0)).alias("revenue")
    )
    .withColumn(
        "conversion_rate",
        F.when(F.col("views") > 0, (F.col("purchases")/F.col("views")*100)).otherwise(0)
    )
)

product_perf.write.format("delta").mode("overwrite").save(gold_path)

print("GOLD completed")


GOLD completed
